<a id="top"></a>
# Object Detection Demo: Car Detection

## OpenVINO version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit. Alternatively, you can open a version of this notebook for the Intel® Distribution of OpenVINO™ Toolkit LTS version by running the cell below and following the link it generates.

In [ ]:
from qarpo import displayMultiversionURL
import os
displayMultiversionURL(os.path.abspath(""), "object_detection_demo.ipynb", "openvino-dev-latest", ['openvino-lts'])

<a id="top"></a>
# Object Detection Demo: Car Detection

This is a sample reference implementation to showcase object detection (car in this case) with single-shot detection (SSD) and Async API.
Async API improves the overall frame-rate of the application by not waiting for the inference to complete but continuing to do things on the host while inference accelerator is busy. 
Specifically, this code demonstrates two parallel inference requests by processing the current frame while the next input frame is being captured. This essentially hides the latency of frame capture.

## Overview of how it works
The inference executable (object-detection-cpp) reads the command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. 
A job is submitted to run the inference executable on a hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Core CPU, Intel® Movidius™ and/or Neural Compute Stick).
After the inference is completed, the output videos are appropriately stored in the /results directory, which can then be viewed within the Jupyter Notebook instance.

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes

## Set Up

### Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import cv2
import os
import time
import sys
from qarpo.demoutils import *
from qarpo.model_visualizer_link import *

## Using OpenVINO

First, let's try running inference on a single image to see how OpenVINO works.
We will be using OpenVINO's Inference Engine (IE) to locate vehicles on the road.
There are five steps involved in this task:

1. Create an Intermediate Representation (IR) Model using the Intel Model Optimizer
2. Choose a device and create IEPlugin for the device
3. Read the IRModel using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### Creating IR Model

Intel Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existing DNN models from popular frameworks (e.g. Caffe, TF) using the Intel Model Optimizer. 

The Intel Distribution of OpenVINO includes a utility script `downloader.py` that you can use to download some common modes. Run the following cell to see the models available through `downloader.py`

In [ ]:
!downloader.py --print_all

**Note** the '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in commannd line. So the above command will work straight out of the box on in a terminal (with '!' removed).

In this demo, we will be using the **mobilenet-ssd** model. This model can be downloaded with the following command.

In [ ]:
!downloader.py --name mobilenet-ssd -o raw_models

The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.


With the `-o` option set as above, this command downloads the model in the directory `raw_models`, with the `.caffemodel` located at `raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel`


To view a graph of the model used in this application, run the cell below then select the link generated:

In [ ]:
showModelVisualizerLink("raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel")

Now, let's convert this to the optimized model using the model optimizer.

In [ ]:
!mo.py \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--data_type FP32 \
-o models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

**Note** the above line is a single command line input, which spans 4 lines thanks to the backslash '\\', which is a line continuation character in Bash.

Here, the arguments are:
* --input-model : the original model
* --data_type : Data type to use. One of {FP32, FP16, half, float}
* -o : output dirctory

This script also supports `-h` that will you can get the full list of arguments.

With the `-o` option set as above, this command will write the output to the directory `models/mobilenet-ssd/FP32`

There are two files produced:
```
models/mobilenet-ssd/FP32/mobilenet-ssd.xml
models/mobilenet-ssd/FP32/mobilenet-ssd.bin
```
These will be used later in the exercise.

We will also be needing the FP16 version of the model for the calculations on the MYRIAD architecture. Run the following cell to create it.

In [ ]:
!mo.py \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--data_type FP16 \
-o models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

### Compile the code
We have provided a CMakefile for compiling the examples. Run the following cell to compile the application.
(tip: use **crtl+enter** to run the cell)

In [ ]:
!mkdir -p build && cd build

!cmake .

!make

### Command line flags

Run the following cells to see the list of the available arguments: 

In [ ]:
!./build/intel64/object-detection-cpp -h

## Running the inference

Now we are ready to run the inference workload. We will run the workload on several edge compute nodes represented in the IoT DevCloud. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

**Note**: Currently, you are running this Notebook on a development node. On this system, you are alloated just one core on a large Xeon CPU. The purpose of this node is to develop code and run minimal jupyter notebooks, but it is not meant for compute jobs like deep learning inference. So we need to request additional resources from the cluster to run the inference, and this is done through the job queue.


The job file is written in Bash, and will be executed directly on the edge compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "object_detection_job.sh" In this step, we will be submitting the workload as a job to the job queue. 

In [ ]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE
ROIFILE=$OUTPUT_FILE/ROIs.txt
OVIDEO=$OUTPUT_FILE/output.bmp

if [ "$FP_MODEL" = "FP32" ]; then
    config_file="conf_fp32.txt"
else
    config_file="conf_fp16.txt"
fi

# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
./build/intel64/object-detection-cpp -i /opt/intel/openvino/deployment_tools/demo/car_1.bmp \
            -m models/mobilenet-ssd/$FP_MODEL/mobilenet-ssd.xml \
            -d $DEVICE \
            -o $OUTPUT_FILE

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the `qsub` command.
We can submit object_detection_job to 5 different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- `-l` : this option lets us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option lets us send arguments to the bash script. 
- `-N` : this option lets us name the job so that it is easier to distinguish between them.

If you are curious to see the available types of nodes on the IoT DevCloud, run the following optional cell.

In [ ]:
!pbsnodes | grep compnode | sort | uniq -c

The `-F` flag is used to pass in arguments to the job script.
The [object_detection_job.sh](object_detection_job.sh) takes in 3 arguments:
1. the path to the directory for the output video and performance stats
2. targeted device (e.g. CPU,GPU,MYRIAD)
3. the floating precision to use for inference


The job scheduler will use the contents of `-F` flag as the argument to the job script.


Finally, the `-N` flag is used to name the job itself. 
By default the jobs take on the name of the job script, which in this case would be "object_detection_job.sh".
But because we are submitting these jobs with different arguments, it is useful for record-keeping to name the job differently based on the arguments.

The following line will request an Intel Core CPU system, passes in "results/core CPU FP32" to the job script, and names the job "obj_det_core". Run the cell to submit this job. 

#### Submitting to a node with Intel® Core™ CPU

In [ ]:
print("Submitting a job to an edge compute node with an Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:idc001skl -F "results/core CPU FP32" -N obj_det_core

#### submitting to a node with Intel® Xeon CPU

In [ ]:
print("Submitting a job to an edge compute node with an Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub object_detection_job.sh -l nodes=1:idc007xv5 -F "results/xeon CPU FP32" -N obj_det_xeon

#### submitting to a node with Intel® Core CPU and using the onboard Intel GPU

In [ ]:
print("Submitting a job to an edge compute node with an Intel Core CPU and an Intel GPU...")
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=1:idc001skl -F "results/gpu GPU FP32" -N obj_det_gpu

#### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2

In [ ]:
print("Submitting job to an edge compute node with Intel NCS2...")
#Submit job to the queue
job_id_ncs2 = !qsub object_detection_job.sh -l nodes=1:idc004nc2 -F "results/ncs2 MYRIAD FP16" -N obj_det_ncs2

#### UP Squared Grove IoT Development Kit

In [ ]:
print("Submitting job to an edge compute node with Intel UP2...")
#Submit job to the queue
job_id_up2 = !qsub object_detection_job.sh -l nodes=1:idc008u2g -F "results/up2 GPU FP32" -N obj_det_up2

### Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up. 

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete. 

***Wait!***

Please wait for the inference jobs and video rendering complete before proceeding to the next step.

## View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`obj_det_{type}.o{JobID}`

`obj_det_{type}.e{JobID}`

(here, obj_det_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output image files. They are stored in .bmp format inside the `results/` directory.
displayImage() prints the output image. 

In [ ]:
def displayImage(image_path):
    plt.figure(figsize = (7,7))
    image = cv2.imread(image_path)
    im_to_show = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(im_to_show)

In [ ]:
image_path = f'results/core/out_0_{job_id_core[0]}.bmp'
displayImage(image_path)

In [ ]:
image_path = f'results/xeon/out_0_{job_id_xeon[0]}.bmp'
displayImage(image_path)

In [ ]:
image_path = f'results/gpu/out_0_{job_id_gpu[0]}.bmp'
displayImage(image_path)

In [ ]:
image_path = f'results/ncs2/out_0_{job_id_ncs2[0]}.bmp'
displayImage(image_path)

In [ ]:
image_path = f'results/up2/out_0_{job_id_up2[0]}.bmp'
displayImage(image_path)

## Assess Performance

The running time of each inference task is recorded in `results/*/stats_{job_id}.txt`, where the subdirectory name corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('ncs2', 'Intel\nNCS2'),
             ('up2', 'Intel\nUP2')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append((f'results/{arch}/stats_{vars()["job_id_"+arch][0]}.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>